In [1]:
import pickle 
import numpy as np 
import dgl
import torch
from scipy.sparse import csr_matrix

with open('./datasets/nasbench201/pkl/desktop-cpu-core-i7-7820x-fp32.pkl', 'rb') as f:
    df = pickle.load(f)
df[9600][:-1]

array([3., 4., 2., 2., 4., 2.])

In [22]:
from modules.tokenizer import tokenizer 
from data_provider.create_latency import *
from scipy.sparse import csr_matrix
import dgl 

def info2mat(arch_str):
    ops = {'input':0, 'nor_conv_1x1':1, 'nor_conv_3x3':2, 'avg_pool_3x3':3, 'skip_connect':4, 'none':5, 'output':6}
    adj_mat = np.array([[0, 1, 1, 0, 1, 0, 0, 0],
                        [0, 0, 0, 1, 0, 1 ,0 ,0],
                        [0, 0, 0, 0, 0, 0, 1, 0],
                        [0, 0, 0, 0, 0, 0, 1, 0],
                        [0, 0, 0, 0, 0, 0, 0, 1],
                        [0, 0, 0, 0, 0, 0, 0, 1],
                        [0, 0, 0, 0, 0, 0, 0, 1],
                        [0, 0, 0, 0, 0, 0, 0, 0]])

    nodes = ['input']
    steps = arch_str.split('+')
    steps_coding = ['0', '0', '1', '0', '1', '2']
    cont = 0
    for step in steps:
        step = step.strip('|').split('|')
        for node in step:
            n, idx = node.split('~') #n: operation, idx: previous node
            assert idx == steps_coding[cont]
            cont += 1
            nodes.append(n)
    nodes.append('output')

    node_mat =np.zeros([8, len(ops)]).astype(int)
    ops_idx = [ops[k] for k in nodes]
    node_mat[[0,1,2,3,4,5,6,7], ops_idx] = 1
    return ops_idx

from data_provider.create_latency import get_arch_str_from_arch_vector
key = df[9600][:-1]
arch_str = get_arch_str_from_arch_vector(key)
graph, one_hot_idx = get_matrix_and_ops(key)
ops_idx = info2mat(arch_str)
dx = dr = dp = 10
token = tokenizer(ops_idx, graph, dx, dr, dp, 'nerf')
ops_idx, token.shape

([0, 2, 3, 1, 1, 3, 1, 6], torch.Size([9, 60]))

In [24]:
graph

array([[0, 1, 1, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0]])

In [19]:
graph = dgl.from_scipy(csr_matrix(graph))
graph = dgl.to_bidirected(graph)
features = torch.tensor(ops_idx).long()
op_idx = torch.argmax(features, dim=1)

DGLError: Expect the number of rows to be the same as the number of columns for sp_mat, got 8 and 7.

In [ ]:

key = df[0][:-1]
graph, one_hot_idx = get_matrix_and_ops(key)
# graph, op_idx = get_adjacency_and_features(graph, one_hot_idx)
graph

array([[0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0]])

In [17]:
graph.shape, op_idx.shape

((9, 9), (9, 6))

In [10]:
import torch
import numpy as np
# 读取文件
data = torch.load('datasets/nasbench201/others/all_nasbench201.pt', weights_only=True)

In [11]:
for i in range(len(data)):
    if data[i]['ops'][1:-1] == [3, 4, 2, 2, 4, 2]:
        print(i)
        break
data[i], 

11934


({'index': 11934,
  'adj': [[0, 1, 1, 0, 1, 0, 0, 0],
   [0, 0, 0, 1, 0, 1, 0, 0],
   [0, 0, 0, 0, 0, 0, 1, 0],
   [0, 0, 0, 0, 0, 0, 1, 0],
   [0, 0, 0, 0, 0, 0, 0, 1],
   [0, 0, 0, 0, 0, 0, 0, 1],
   [0, 0, 0, 0, 0, 0, 0, 1],
   [0, 0, 0, 0, 0, 0, 0, 0]],
  'ops': [0, 3, 4, 2, 2, 4, 2, 6],
  'training_time': 131.4312922358513,
  'test_accuracy': 74.19,
  'test_accuracy_avg': 74.19,
  'valid_accuracy': 74.47600001220704,
  'valid_accuracy_avg': 74.47600001220704,
  'netcode': tensor([[ 0.0000e+00,  1.0000e+00,  0.0000e+00,  ...,  9.9989e-01,
            6.8643e-01,  7.2720e-01],
          [-2.3850e-08, -1.0000e+00,  5.2630e-01,  ...,  1.0000e+00,
            0.0000e+00,  1.0000e+00],
          [ 3.4969e-07,  1.0000e+00, -9.7689e-01,  ...,  1.0000e+00,
            0.0000e+00,  1.0000e+00],
          ...,
          [ 1.7485e-07,  1.0000e+00,  6.2701e-01,  ...,  1.8229e-02,
           -1.8406e-01,  6.3810e-01],
          [ 4.7700e-08,  1.0000e+00,  8.9502e-01,  ...,  9.2416e-01,
        